In [89]:
import networkx as nx

import matplotlib.pyplot as plt

# Create a 3D grid graph
G = nx.grid_graph(dim=[8, 8, 8])



In [90]:
import numpy as np

import plotly.graph_objects as go

# Define the number of points in each dimension
num_points = 8

# Generate evenly spaced coordinates
x_coords = np.linspace(-1, 1, num_points)
y_coords = np.linspace(-1, 1, num_points)
z_coords = np.linspace(-1, 1, num_points)

# Create a meshgrid of coordinates
X, Y, Z = np.meshgrid(x_coords, y_coords, z_coords)

# Reshape the coordinates into a 1D array
x_coords = X.flatten()
y_coords = Y.flatten()
z_coords = Z.flatten()

# Create a dictionary to store the coordinates
pos = {}
for i in range(len(x_coords)):
    pos[i] = np.array([x_coords[i], y_coords[i], z_coords[i]])

# Create a scatter3d trace
trace = go.Scatter3d(
    x=[node[0] for node in G.nodes()],
    y=[node[1] for node in G.nodes()],
    z=[node[2] for node in G.nodes()],
    mode='markers',
    marker=dict(
        size=5,
        color='blue'
    )
)

# Create the layout
layout = go.Layout(
    title='3D Graph',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    )
)


# Create the figure
fig = go.Figure(data=[trace], layout=layout)


# Create a list to store the edge traces
edge_traces = []

# Iterate over each edge in the graph
for edge in G.edges:
    # Get the start and end coordinates of the edge
    start = edge[0]
    end = edge[1]
    
    # Create a scatter3d trace for the edge
    edge_trace = go.Scatter3d(
        x=[start[0], end[0]],
        y=[start[1], end[1]],
        z=[start[2], end[2]],
        mode='lines',
        line=dict(color='red', width=2)
    )
    
    # Add the edge trace to the list
    edge_traces.append(edge_trace)

# Add the edge traces to the figure
fig.add_traces(edge_traces)

# Show the figure
#fig.show()

In [91]:
# Create a mapping from node coordinates to unique integer IDs
coord_to_id = {coord: i for i, coord in enumerate(G.nodes())}
id_to_coord = {i: coord for coord, i in coord_to_id.items()}



In [92]:
subgrid_size = (2, 2)
subgrid_to_nodes = {}
node_to_subgrid = {}

for node in G.nodes():
    x, y, z = node
    subgrid_x = x // subgrid_size[0]
    subgrid_y = y // subgrid_size[1]
    subgrid = (subgrid_x, subgrid_y)
    
    if subgrid not in subgrid_to_nodes:
        subgrid_to_nodes[subgrid] = []
    
    subgrid_to_nodes[subgrid].append(node)
    node_to_subgrid[node] = subgrid

In [93]:
import numpy as np

import plotly.graph_objects as go

# create color map for subgrids
colors = [i*50 for i in range(len(subgrid_to_nodes.keys()))]
# dict to store subgrid to color mapping
subgrid_to_color = {}
# iterate over subgrids and assign a color
for i, subgrid in enumerate(subgrid_to_nodes.keys()):
    subgrid_to_color[subgrid] = colors[i]
# Create a scatter3d trace with color based on subgrid

# choose a color palette with random colors

colorpalette = [[i, 'rgb({},{},{})'.format(np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))] for i in np.linspace(0, 1, len(subgrid_to_nodes.keys()))]
trace = go.Scatter3d(
    x=[node[0] for node in G.nodes()],
    y=[node[1] for node in G.nodes()],
    z=[node[2] for node in G.nodes()],
    mode='markers',
    marker=dict(
        size=5,
        color=[subgrid_to_color[node_to_subgrid[node]] for node in G.nodes()],
        colorscale=colorpalette,
        colorbar=dict(title='Subgrid')
    )
)

# Create the layout
layout = go.Layout(
    title='3D Graph',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    )
)

# Create the figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

In [95]:
import copy
from collections import defaultdict
paths = defaultdict(list)
for k, v in id_to_coord.items():
    for i, e in enumerate(v):
        copy_v = list(copy.copy(v))
        copy_v[i] += 1
            
        if copy_v[i] + 1 > 7:
            continue
        
        for j, f in enumerate(copy_v):
            copy_v_2 = list(copy.copy(copy_v))
            copy_v_2[j] += 1
            
            if copy_v_2[j] + 1 > 7:
                continue

            paths[v].append([v, copy_v, copy_v_2])

In [96]:
len(paths)

492

In [97]:
from networkx import all_neighbors
import random
dic = {}
for k, v in paths.items():
    end_nodes = set()
    ret = []
    for path in v:
        start, mid, end = path
        end_nodes.add(tuple(end))
    for path in v:
        tmp = {}
        neighbors = set()
        for node in path: 
            neighbors.update(all_neighbors(G, tuple(node)))
        tmp["path"] = path
        tmp["neighbors"] = neighbors

        possible_samples = end_nodes - neighbors
        if len(possible_samples) > 0:
            special_distractor = random.choice(list(possible_samples))
        else:
            continue
        tmp["special_distractor"] = special_distractor
        ret.append(tmp)
    dic[k] = ret


In [98]:
len(dic[(0,0,0)][0])

3

In [99]:
dic[(0,0,0)][2]

{'path': [(0, 0, 0), [1, 0, 0], [1, 0, 1]],
 'neighbors': {(0, 0, 0),
  (0, 0, 1),
  (0, 1, 0),
  (1, 0, 0),
  (1, 0, 1),
  (1, 0, 2),
  (1, 1, 0),
  (1, 1, 1),
  (2, 0, 0),
  (2, 0, 1)},
 'special_distractor': (0, 2, 0)}

In [100]:
len(dic)

492

In [101]:
dic

{(0,
  0,
  0): [{'path': [(0, 0, 0), [1, 0, 0], [2, 0, 0]],
   'neighbors': {(0, 0, 0),
    (0, 0, 1),
    (0, 1, 0),
    (1, 0, 0),
    (1, 0, 1),
    (1, 1, 0),
    (2, 0, 0),
    (2, 0, 1),
    (2, 1, 0),
    (3, 0, 0)},
   'special_distractor': (0, 2, 0)}, {'path': [(0, 0, 0),
    [1, 0, 0],
    [1, 1, 0]],
   'neighbors': {(0, 0, 0),
    (0, 0, 1),
    (0, 1, 0),
    (1, 0, 0),
    (1, 0, 1),
    (1, 1, 0),
    (1, 1, 1),
    (1, 2, 0),
    (2, 0, 0),
    (2, 1, 0)},
   'special_distractor': (0, 1, 1)}, {'path': [(0, 0, 0),
    [1, 0, 0],
    [1, 0, 1]],
   'neighbors': {(0, 0, 0),
    (0, 0, 1),
    (0, 1, 0),
    (1, 0, 0),
    (1, 0, 1),
    (1, 0, 2),
    (1, 1, 0),
    (1, 1, 1),
    (2, 0, 0),
    (2, 0, 1)},
   'special_distractor': (0, 2, 0)}, {'path': [(0, 0, 0),
    [0, 1, 0],
    [1, 1, 0]],
   'neighbors': {(0, 0, 0),
    (0, 0, 1),
    (0, 1, 0),
    (0, 1, 1),
    (0, 2, 0),
    (1, 0, 0),
    (1, 1, 0),
    (1, 1, 1),
    (1, 2, 0),
    (2, 1, 0)},
   'special_dist

In [102]:
# distractory jen ze stejného modulu
# special distractor musí být z toho stejného modulu, pokud tam je jinak none
# to stejné u subgoal a finalgoal
# {id_modulu: {random_distractors: set(), special_distractor: id/none, subgoal: id/none, finalgoal: id/none}}

In [104]:
import random

training_examples = []
testing_examples = []
for node in dic:
    paths = dic[node]
    for sample in paths[:8]:
        for i in range(50):
            training = {}
            for submodule, sub_nodes in subgrid_to_nodes.items():
                final_goal, sub_goal, spec_dis = tuple(sample["path"][2]), tuple(sample["path"][1]), sample["special_distractor"]
                # podívám se do kterého modulu patří jednotlivé prvky
                # podívám se jestli spadá do current submodulu, pokud je tak nechám to ID jak je a jinak dám None
                temp2 = {
                    "final_goal": final_goal if final_goal in sub_nodes else None,
                    "sub_goal": sub_goal if sub_goal in sub_nodes else None,
                    "special_distractor": spec_dis if spec_dis in sub_nodes else None
                }
                # přidat random distractors - vezmu všechny ID nodů z tohohle modulu a vezmu rozdíl mezi tímhle a neighbours
                # vysamplovat 4 náhodné a přidat jako "random_distractors" do temp2
                all_nodes = set(sub_nodes)
                neighbours = set(sample["neighbors"])
                available_distractors = list(all_nodes - neighbours - {final_goal, sub_goal, spec_dis})
                random_distractors = random.sample(available_distractors, min(4, len(available_distractors)))
                temp2["random_distractors"] = random_distractors
                
                training[submodule] = temp2
            training["target"] = sample["path"][0]
            training_examples.append(training)
    
    if len(paths) > 8: 
        sample = paths[8]
        testing = {}
        for submodule, sub_nodes in subgrid_to_nodes.items():
            final_goal, sub_goal, spec_dis = tuple(sample["path"][2]), tuple(sample["path"][1]), sample["special_distractor"]
            # podívám se do kterého modulu patří jednotlivé prvky
            # podívám se jestli spadá do current submodulu, pokud je tak nechám to ID jak je a jinak dám None
            temp2 = {
                "final_goal": final_goal if final_goal in sub_nodes else None,
                "sub_goal": sub_goal if sub_goal in sub_nodes else None,
                "special_distractor": spec_dis if spec_dis in sub_nodes else None
            }
            # přidat random distractors - vezmu všechny ID nodů z tohohle modulu a vezmu rozdíl mezi tímhle a neighbours
            # vysamplovat 4 náhodné a přidat jako "random_distractors" do temp2
            all_nodes = set(sub_nodes)
            neighbours = set(sample["neighbors"])
            available_distractors = list(all_nodes - neighbours - {final_goal, sub_goal, spec_dis})
            random_distractors = random.sample(available_distractors, min(4, len(available_distractors)))
            temp2["random_distractors"] = random_distractors
            
            testing[submodule] = temp2
        testing["target"] = sample["path"][0]
        testing_examples.append(testing)

In [105]:
len(training_examples)

118550

In [106]:
    # z toho už sekvence
    # pojedu přes všechny submoduly, vezmu dict z submodulu a vytvořím podsekvenci, nakonec konkatenuju, stejné pořadí.
    # každá podsekvence len = 4
    # random distractors bude max 4, pokud tam bude jedno z final_goal, sub_goal nebo special_distractor tak podle toho doplnit max 4
    # v rámci podsekvence shuffle
    # target bude poslední vždycky
    # subgoal, spec. distractor, finalgoal musíme vědět pozice (modul, pozice)

In [107]:
test_data = []
for sample_dic in testing_examples:
    sequence = []
    positions = {}
    for k, module_data in sample_dic.items():
        if k == 'target':
            target = module_data
            continue
        elif isinstance(module_data, dict):
            final_goal = module_data.get('final_goal', None)
            sub_goal = module_data.get('sub_goal')
            special_distractor = module_data.get('special_distractor')
            random_distractors = module_data.get('random_distractors')

        subsequence = []

        goal_id = None
        sub_goal_id = None
        sd_id = None

        if final_goal is not None:
            goal_id = coord_to_id[final_goal]
        if sub_goal is not None:
            sub_goal_id = coord_to_id[sub_goal]
        if special_distractor is not None:
            sd_id = coord_to_id[special_distractor]

        subsequence = [goal_id] + [sub_goal_id] + [sd_id] + [None]

        for i, sample in enumerate(subsequence):
            if sample is None and random_distractors:
                to_swap = random.choice(random_distractors)
                subsequence[i] = coord_to_id[to_swap]
        random.shuffle(subsequence)

        if goal_id is not None:
            positions['final_goal'] = (k, subsequence.index(goal_id))
        if sub_goal_id is not None:
            positions['sub_goal'] = (k, subsequence.index(sub_goal_id))
        if sd_id is not None:
            positions['special_distractor'] = (k, subsequence.index(sd_id))

        sequence.extend(subsequence)
    
    sequence.append(coord_to_id[target])

    # Calculate the actual indices
    actual_positions = {}
    positions_extended = {}
    for key, (module, subindex) in positions.items():
        actual_index = list(sample_dic.keys()).index(module) * 4 + subindex
        actual_positions[key] = actual_index
        positions_extended[key] = actual_index + 513

    test_data.append({
            'input': sequence,
            'positions': actual_positions,
            'positions_extended': positions_extended,
            'last_idx': len(sequence) - 1,
            'last_idx_extended': len(sequence) - 1 + 513
        })

In [108]:
test_data[25]

{'input': [15,
  66,
  65,
  15,
  28,
  92,
  19,
  19,
  100,
  40,
  40,
  40,
  113,
  113,
  120,
  113,
  137,
  192,
  142,
  142,
  214,
  147,
  214,
  214,
  237,
  237,
  169,
  171,
  185,
  185,
  249,
  180,
  270,
  270,
  270,
  328,
  282,
  277,
  277,
  277,
  359,
  359,
  301,
  359,
  309,
  374,
  374,
  305,
  399,
  385,
  385,
  399,
  471,
  405,
  405,
  470,
  425,
  428,
  488,
  428,
  504,
  501,
  445,
  511,
  64],
 'positions': {'final_goal': 1, 'sub_goal': 2, 'special_distractor': 17},
 'positions_extended': {'final_goal': 514,
  'sub_goal': 515,
  'special_distractor': 530},
 'last_idx': 64,
 'last_idx_extended': 577}

In [109]:
print(id_to_coord[66], id_to_coord[65], id_to_coord[80])

(1, 0, 2) (1, 0, 1) (1, 2, 0)


In [110]:
train_data = []
for sample_dic in training_examples:
    sequence = []
    positions = {}
    for k, module_data in sample_dic.items():
        if k == 'target':
            target = module_data
            continue
        elif isinstance(module_data, dict):
            final_goal = module_data.get('final_goal', None)
            sub_goal = module_data.get('sub_goal')
            special_distractor = module_data.get('special_distractor')
            random_distractors = module_data.get('random_distractors')

        subsequence = []

        goal_id = None
        sub_goal_id = None
        sd_id = None

        if final_goal is not None:
            goal_id = coord_to_id[final_goal]
        if sub_goal is not None:
            sub_goal_id = coord_to_id[sub_goal]
        if special_distractor is not None:
            sd_id = coord_to_id[special_distractor]

        subsequence = [goal_id] + [sub_goal_id] + [sd_id] + [None]

        for i, sample in enumerate(subsequence):
            if sample is None and random_distractors:
                to_swap = random.choice(random_distractors)
                subsequence[i] = coord_to_id[to_swap]
        random.shuffle(subsequence)

        if goal_id is not None:
            positions['final_goal'] = (k, subsequence.index(goal_id))
        if sub_goal_id is not None:
            positions['sub_goal'] = (k, subsequence.index(sub_goal_id))
        if sd_id is not None:
            positions['special_distractor'] = (k, subsequence.index(sd_id))

        sequence.extend(subsequence)
    
    sequence.append(coord_to_id[target])

    # Calculate the actual indices
    actual_positions = {}
    positions_extended = {}
    for key, (module, subindex) in positions.items():
        actual_index = list(sample_dic.keys()).index(module) * 4 + subindex
        actual_positions[key] = actual_index
        positions_extended[key] = actual_index + 513

    train_data.append({
            'input': sequence,
            'positions': actual_positions,
            'positions_extended': positions_extended,
            'last_idx': len(sequence) - 1,
            'last_idx_extended': len(sequence) - 1 + 513
        })

In [114]:
train_data

[{'input': [64,
   76,
   11,
   76,
   87,
   16,
   31,
   94,
   105,
   97,
   105,
   33,
   114,
   49,
   117,
   49,
   133,
   133,
   128,
   132,
   209,
   209,
   153,
   212,
   225,
   225,
   225,
   227,
   178,
   178,
   178,
   255,
   256,
   321,
   324,
   321,
   277,
   273,
   273,
   277,
   292,
   292,
   292,
   361,
   314,
   319,
   319,
   314,
   389,
   385,
   389,
   463,
   401,
   478,
   414,
   401,
   421,
   485,
   424,
   483,
   506,
   506,
   446,
   446,
   0],
  'positions': {'sub_goal': 0, 'special_distractor': 5, 'final_goal': 18},
  'positions_extended': {'sub_goal': 513,
   'special_distractor': 518,
   'final_goal': 531},
  'last_idx': 64,
  'last_idx_extended': 577},
 {'input': [64,
   11,
   73,
   11,
   16,
   92,
   90,
   17,
   100,
   102,
   100,
   99,
   126,
   62,
   50,
   126,
   130,
   128,
   130,
   195,
   156,
   208,
   156,
   159,
   173,
   164,
   175,
   162,
   186,
   249,
   249,
   176,
   328,
   33